In [ ]:
from google.colab import drive
drive.mount('/content/drive')
#/content/drive/MyDrive/

Mounted at /content/drive


# Import Thư viện

In [ ]:
!pip install underthesea -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.9/20.9 MB 44.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 657.8/657.8 kB 28.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 40.7 MB/s eta 0:00:00


In [ ]:
!pip install pyvi -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 25.5 MB/s eta 0:00:00


In [ ]:
import re
import pandas as pd
from underthesea import pos_tag
import random
from pyvi import ViTokenizer, ViPosTagger  # Giả sử bạn sử dụng pyvi để phân tích ngữ pháp

In [1]:
# Đọc dữ liệu từ file CSV hoặc JSON
df = pd.read_csv('/content/drive/MyDrive/ChatBot/data/processed_medical_cleaned.csv')  # Thay đổi đường dẫn đến tập dữ liệu của bạn

# Kiểm tra dữ liệu
df.head()




NameError: name 'pd' is not defined

In [ ]:
# Hàm để chia văn bản thành các câu, bỏ qua câu bắt đầu bằng "Ví dụ" và các câu ngắn
def split_into_sentences(text, min_length=20):
    # Kiểm tra xem text có phải là chuỗi không và xử lý NaN
    if isinstance(text, str):
        # Tách văn bản theo dấu chấm câu như ".", "!", "?" và giữ lại dấu câu
        sentences = re.split(r'(?<=[.!?]) +', text)
        # Loại bỏ câu bắt đầu bằng "ví_dụ" và các câu ngắn hơn độ dài tối thiểu
        filtered_sentences = [sentence for sentence in sentences
                              if not sentence.strip().startswith("ví_dụ")
                              and len(sentence.strip()) > min_length]
        return filtered_sentences
    else:
        # Trả về danh sách rỗng nếu text không phải là chuỗi hoặc là NaN
        return []

In [ ]:
df.shape

(11892, 6)

# Pos (NLP – Part of Speech)

In [ ]:
def load_top_words(filepath):
    with open(filepath, 'r', encoding='utf-8') as f:
        top_words = set([line.strip() for line in f])  # Đọc từng dòng và lưu vào một tập hợp
    return top_words

# Đọc danh sách các từ từ file top_word.txt
top_words = load_top_words('/content/drive/MyDrive/ChatBox/data/vietnamese-stopwords-dash.txt')


In [ ]:
import nltk
from nltk.corpus import words

# Tải danh sách từ tiếng Anh từ NLTK
nltk.download('words')
english_words = set(words.words())

[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!


In [ ]:
# def remove_english_keywords(keywords):
#     filtered_keywords = {}

#     for key in keywords:
#         filtered_keywords[key] = [word for word in keywords[key] if word not in english_words]

#     return filtered_keywords

In [ ]:
def extract_keywords(sentence, top_words_file):
    # Tiền xử lý câu để loại bỏ các ký tự không cần thiết
    sentence = re.sub(r'[^\w\s]', '', sentence)  # Loại bỏ dấu câu và ký tự đặc biệt

    # Gán nhãn POS cho các từ trong câu
    pos_tags = ViPosTagger.postagging(ViTokenizer.tokenize(sentence))

    # Tạo danh sách chứa từ và nhãn của nó
    tagged_words = [(word, pos) for word, pos in zip(pos_tags[0], pos_tags[1])]

    # Chỉ giữ lại danh từ (N), động từ (V), tính từ (A), đại từ (P)
    keywords = []
    for word, pos in tagged_words:
        if (pos.startswith('N') or pos.startswith('V') or pos.startswith('A') or pos.startswith('P')) and len(word) > 1:
            keywords.append((word, pos))  # Giữ nguyên thứ tự

    # Loại bỏ các từ dừng (stop words)
    stop_words = set(['và', 'hoặc', 'của', 'trong', 'theo', 'là', 'có', 'ví_dụ'])
    keywords = [(word, pos) for word, pos in keywords if word not in stop_words]

    # Loại bỏ các từ có trong top_words
    keywords = [(word, pos) for word, pos in keywords if word not in top_words_file]

    return keywords


In [ ]:
def generate_questions_with_title(keywords, title, top_words, use_keywords_from_title=False):
    templates = {
        'C→Vđ': "Tại sao {noun} lại {verb} trong bối cảnh của {title}?",
        'C→Vq−B': "Những yếu tố nào ảnh hưởng đến {noun} khi {verb} trong {title}?",
        'Tr→Vt−B': "Liệu {noun} có vai trò gì trong {title} không?",
        'xV→yV’': "Nếu {noun} {verb}, điều này có ý nghĩa gì cho {title}?",
        'V→C': "{verb} {noun} có ảnh hưởng như thế nào đến {title}?",
        'C→A': "Tính chất của {noun} có ảnh hưởng đến {title} ra sao?",
        'N→V': "Tại sao {noun} lại cần {verb} trong {title}?",
        'N+N→V': "Sự kết hợp giữa {noun} và {noun} có thể {verb} như thế nào trong {title}?",
        'C→Vh': "Có thể nói gì về việc {noun} {verb} trong {title}?",
        'Tại sao...': "Tại sao {noun} lại quan trọng trong {title}?",
        'V→C+N': "{verb} {title} có liên quan đến {noun} như thế nào?",
    }

    questions = []
    nouns = keywords['nouns']
    verbs = keywords['verbs']
    adjectives = keywords['adjectives']

    # Xử lý tiêu đề
    if use_keywords_from_title:
        title_keywords = extract_keywords(title, top_words)
        title_noun = next((word for word, pos in title_keywords if pos.startswith('N')), title)
        title = title_noun
    else:
        title = title.strip()

    # Lấy từ khóa đầu tiên
    main_noun = nouns[0] if nouns else 'vật chất'
    main_verb = verbs[0] if verbs else 'hỗ trợ'

    # Tạo câu hỏi với từ khóa đầu tiên từ template ngẫu nhiên
    question_template = random.choice(list(templates.values()))
    question = question_template.format(noun=main_noun, verb=main_verb, title=title)
    questions.append(question)

    # Tạo tối đa 2 câu hỏi bổ sung với danh từ và động từ còn lại
    additional_questions = 0
    for i in range(1, min(len(nouns), len(verbs))):
        if additional_questions < 2:  # Giới hạn số lượng câu hỏi bổ sung
            question_template = random.choice(list(templates.values()))
            question = question_template.format(noun=nouns[i], verb=verbs[i], title=title)
            questions.append(question)
            additional_questions += 1

    return questions

In [ ]:
new_rows = []

for index, row in df.iterrows():
    # Chia đoạn văn Detailed Content thành các câu
    sentences = split_into_sentences(row['Detailed Content'])

    # Duyệt qua từng câu trong Detailed Content
    for sentence in sentences:
        # Lấy các từ khóa từ câu
        keywords = extract_keywords(sentence, top_words)

        # Phân loại từ theo POS, kiểm tra kiểu dữ liệu
        nouns = [word for word, pos in keywords if isinstance((word, pos), tuple) and pos.startswith('N')]
        verbs = [word for word, pos in keywords if isinstance((word, pos), tuple) and pos.startswith('V')]
        adjectives = [word for word, pos in keywords if isinstance((word, pos), tuple) and pos.startswith('A')]
        pronouns = [word for word, pos in keywords if isinstance((word, pos), tuple) and pos.startswith('P')]

        # Nối tất cả các từ khóa lại
        all_keywords = nouns + verbs + adjectives + pronouns

        # Tạo câu hỏi từ từ khóa
        questions = generate_questions_with_title({
            'nouns': nouns,
            'verbs': verbs,
            'adjectives': adjectives,
            'pronouns': pronouns
        }, row['Title'], top_words)

        # Tìm vị trí start và end của câu trả lời trong đoạn Detailed Content
        answer_start = row['Detailed Content'].find(sentence)
        answer_end = answer_start + len(sentence)

        # Thêm từng câu hỏi vào DataFrame
        for question in questions:
            new_rows.append({
                'Title': row['Title'],
                'Detailed Content': row['Detailed Content'],
                'Answer': sentence,
                'Keywords': ', '.join(all_keywords),
                'Question': question,
                'Answer_Start': answer_start,
                'Answer_End': answer_end,
                'Reference Link': row['Reference Link']
            })

# Chuyển đổi danh sách các câu thành DataFrame mới
new_df = pd.DataFrame(new_rows)

# Hiển thị DataFrame kết quả
new_df.head()


,Title,Detailed Content,Answer,Keywords,Question,Answer_Start,Answer_End,Reference Link
0,rào_cản tự_nhiên chống lại nhiễm_trùng về da,da thường chặn các vi_sinh_vật xâm_nhập trừ kh...,da thường chặn các vi_sinh_vật xâm_nhập trừ kh...,"da, vi_sinh_vật, động_vật, ống, iv, chặn, xâm_...",Liệu da có vai trò gì trong rào_cản tự_nhiên c...,0,139,https://www.msdmanuals.com/vi-vn/professional/...
1,rào_cản tự_nhiên chống lại nhiễm_trùng về da,da thường chặn các vi_sinh_vật xâm_nhập trừ kh...,da thường chặn các vi_sinh_vật xâm_nhập trừ kh...,"da, vi_sinh_vật, động_vật, ống, iv, chặn, xâm_...",Tại sao vi_sinh_vật lại xâm_nhập trong bối cản...,0,139,https://www.msdmanuals.com/vi-vn/professional/...
2,rào_cản tự_nhiên chống lại nhiễm_trùng về da,da thường chặn các vi_sinh_vật xâm_nhập trừ kh...,da thường chặn các vi_sinh_vật xâm_nhập trừ kh...,"da, vi_sinh_vật, động_vật, ống, iv, chặn, xâm_...",trừ động_vật có ảnh hưởng như thế nào đến rào_...,0,139,https://www.msdmanuals.com/vi-vn/professional/...
3,rào_cản tự_nhiên chống lại nhiễm_trùng về da,da thường chặn các vi_sinh_vật xâm_nhập trừ kh...,các ngoại_lệ bao_gồm virus papilloma_người có_...,"ngoại_lệ, virus, papilloma_người, da, mụn, cóc...",Sự kết hợp giữa ngoại_lệ và ngoại_lệ có thể ba...,140,479,https://www.msdmanuals.com/vi-vn/professional/...
4,rào_cản tự_nhiên chống lại nhiễm_trùng về da,da thường chặn các vi_sinh_vật xâm_nhập trừ kh...,các ngoại_lệ bao_gồm virus papilloma_người có_...,"ngoại_lệ, virus, papilloma_người, da, mụn, cóc...",Tại sao virus lại cần xâm_nhập trong rào_cản t...,140,479,https://www.msdmanuals.com/vi-vn/professional/...


In [ ]:
new_df.shape

(164289, 8)

In [ ]:
# Lưu DataFrame vào file CSV
new_df.to_csv('/content/drive/MyDrive/ChatBox/data/processed_medical_full.csv', index=False, encoding='utf-8')